#### dash_bootstrap_components = Cooked up HTML, CSS code
##### https://dash-bootstrap-components.opensource.faculty.ai/docs/components/

#### Dash = Library for plotting, using Python
##### https://dash.plotly.com/dash-core-components/graph

In [ ]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Output, Input
import plotly
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
from datetime import datetime

In [ ]:
# Constants
external_stylesheets = [dbc.themes.BOOTSTRAP, 'https://cdn.plot.ly/plotly-basic-2.12.1.min.js']
app = Dash(__name__, external_stylesheets=external_stylesheets)
colors = ['rgba(31, 74, 194,0.7)', 'rgb(31, 194, 72)', 'rgb(255, 0, 0)', 'rgb(255, 0, 0)', 'rgb(255, 0, 0)','rgb(255, 0, 0)', 'rgb(255, 0, 0)','rgb(255, 0, 0)']

# Storing all the attack labels and features, for later use
names = ['MITM_UNALTERED','NORMAL','RESPONSE_ATTACK','FORCE_ERROR_ATTACK','RECOGNITION_ATTACK','WRITE_ATTACK','READ_ATTACK','REPLAY_ATTACK']
params = ['Time', 'smac', 'dmac', 'sip', 'dip', 'request', 'fc', 'error','address', 'data']

#### Read the d.csv in to a dataframe

In [ ]:
# Read the initial dataframe
df = pd.read_csv('d.csv')

# Constants for all graphs
groups = None
l = None
marker = None
color = None

#### Preparing a layout grid (Rows and columns), to add the graphs and data to

![](https://drive.google.com/uc?export=view&id=1ipGCPSUqY1qNL_NtPRgCZ-1A9GxeRByG)

In [ ]:
# Render the HTML Grid
app.layout = html.Div( # Div containing the Row-Col structure for the dashboard
        [
            html.Br(), # HTML tags in DBC
            dbc.Row([
                dbc.Col(dbc.Row([
                    html.B(id='time', style={'textAlign': 'end'}), html.I(children='Samruddhi K', style={'textAlign': 'end'})
                ]), width=1),
                dbc.Col(html.H2(children="Anomaly Detection in ICPS simulated using the Electra Dataset", style={'textAlign': 'center'}), width=11)
            ]),
            dbc.Row( # This is the stucture using placeholders for each graph component, identified with an id, the actual graph code is defined below.
                [
                    dbc.Col(dcc.Graph(id='addr', animate=True), width=4), # Graph components(defined below) in Columns
                    dbc.Col(dcc.Graph(id='req', animate=True), width=4), # Bootstrap screen width = 12 row units
                    dbc.Col(dcc.Graph(id='err', animate=True), width=4),
                ],
                className="g-0", # Remove gutter (horizontal padding)
            ),
            dbc.Row(
                [
                    dbc.Col([
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='data', animate=True), width=6),
                            dbc.Col(dcc.Graph(id='fc', animate=True), width=6),
                        ], className="g-0"),
                        html.Br(),
                        dbc.Row(
                            dbc.Col(width={'offset':1}, id='prediction'),
                        )
                    ], width=8),
                    dbc.Col(
                        dbc.Row([html.H6(children='Parameters of the latest data point', style={'textAlign': 'center'}),
                            html.Div(id='table')
                        ]),
                        style={'marginTop': '10px', 'fontSize':13}, width=4)
                ],
                className="g-0",
            ),
            dcc.Interval(
                id='interval-component',
                interval=3*1000, # milliseconds - Reload the graphs after 3 seconds
            )
        ],
    )

#### Each component is refreshed after the above said interval - 3 seconds

In [ ]:
# A decorator for the time component of the dashboard - uses the defined inteval as the input and outputs time to the children attribute of html component having id = time.
@app.callback([Output('time','children')], [Input('interval-component', 'n_intervals')])
def show_time(n):
    # Show Current Time
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return [current_time]

In [ ]:
# Update the graph component, having id = addr's figure attribute
@app.callback([Output('addr','figure')], [Input('interval-component', 'n_intervals')])
def g1(n):
    global df, groups, l, marker, color

    # Read d.csv into a dataframe (updated every 3 seconds as per set interval)
    df = pd.read_csv('d.csv')
    groups = df.groupby('label') # Group points by the attack label, for similar colors

    # Getting the last record from the dataframe of d.csv (latest data point)
    l = df.tail(1)

    # Color the latest data point as per the label
    color = 'rgba(31, 74, 194,0.7)' if list(l['label'])[0] == 'MITM_UNALTERED' else 'rgb(31, 194, 72)' if list(l['label'])[0] == 'NORMAL' else 'rgb(255, 0, 0)'

    # Define a marker / border to the latest data point
    marker = dict(line=dict(color='black',width=3), color=color, size=20)

    # Plot Address vs Time graph for the updated dataframe (all remaining points plotted, using scatter plot), using above defined colors
    x = 'Address'
    data = []
    for group,dataframe in groups:
        i = names.index(group)
        trace = go.Scatter(x=dataframe.Time.tolist(), y=dataframe[x.lower()].tolist(), marker=dict(color=colors[i], size=10), mode='lines+markers', name=group)
        data.append(trace)

    # Add the latest point's marker to the plot / trace
    trace = go.Scatter(x=[list(l['Time'])[0]], y=[list(l['address'])[0]], marker=marker, mode='markers', name='Latest')
    data.append(trace)

    # Draw the graph component and return it
    layout = go.Layout(xaxis={'title': 'Time'}, yaxis={'title': x.lower()}, width=450, height=250, showlegend=(False if x!='Error' else True), margin=dict(r=0, t=10, b=10))
    figure = go.Figure(data=data, layout=layout)
    return [figure]

#### Similarly, 4 other features are plotted vs Time

In [ ]:
@app.callback([Output('req','figure')], [Input('interval-component', 'n_intervals')])
def g2(n):
    global df, groups, l, marker, color

    # Plot graph-2 for the updated dataframe
    x = 'Request'
    data = []
    for group,dataframe in groups:
        i = names.index(group)
        trace = go.Scatter(x=dataframe.Time.tolist(), y=dataframe[x.lower()].tolist(), marker=dict(color=colors[i], size=10), mode='lines+markers', name=group)
        data.append(trace)
    trace = go.Scatter(x=[list(l['Time'])[0]], y=[list(l['request'])[0]], marker=marker, mode='markers', name='Latest')
    data.append(trace)
    layout = go.Layout(xaxis={'title': 'Time'}, yaxis={'title': x.lower()}, width=450, height=250, showlegend=(False if x!='Error' else True), margin=dict(r=0, t=10, b=10))
    figure = go.Figure(data=data, layout=layout)
    return [figure]

@app.callback([Output('err','figure')], [Input('interval-component', 'n_intervals')])
def g3(n):
    global df, groups, l, marker, color

    # Plot graph-3 for the updated dataframe
    x = 'Error'
    data = []
    for group,dataframe in groups:
        i = names.index(group)
        trace = go.Scatter(x=dataframe.Time.tolist(), y=dataframe[x.lower()].tolist(), marker=dict(color=colors[i], size=10), mode='lines+markers', name=group)
        data.append(trace)
    trace = go.Scatter(x=[list(l['Time'])[0]], y=[list(l['error'])[0]], marker=marker, mode='markers', name='Latest')
    data.append(trace)
    layout = go.Layout(xaxis={'title': 'Time'}, yaxis={'title': x.lower()}, width=450, height=250, showlegend=(False if x!='Error' else True), margin=dict(r=0, t=10, b=10))
    figure = go.Figure(data=data, layout=layout)
    return [figure]

@app.callback([Output('data','figure')], [Input('interval-component', 'n_intervals')])
def g4(n):
    global df, groups, l, marker, color

    # Plot graph-4 for the updated dataframe
    x = 'Data'
    data = []
    for group,dataframe in groups:
        i = names.index(group)
        trace = go.Scatter(x=dataframe.Time.tolist(), y=dataframe[x.lower()].tolist(), marker=dict(color=colors[i], size=10), mode='lines+markers', name=group)
        data.append(trace)
    trace = go.Scatter(x=[list(l['Time'])[0]], y=[list(l['data'])[0]], marker=marker, mode='markers', name='Latest')
    data.append(trace)
    layout = go.Layout(xaxis={'title': 'Time'}, yaxis={'title': x.lower()}, width=450, height=250, showlegend=(False if x!='Error' else True), margin=dict(r=0, t=10, b=10))
    figure = go.Figure(data=data, layout=layout)
    return [figure]

@app.callback([Output('fc','figure')], [Input('interval-component', 'n_intervals')])
def g5(n):
    global df, groups, l, marker, color

    # Plot graph-5 for the updated dataframe
    x = 'FC'
    data = []
    for group,dataframe in groups:
        i = names.index(group)
        trace = go.Scatter(x=dataframe.Time.tolist(), y=dataframe[x.lower()].tolist(), marker=dict(color=colors[i], size=10), mode='lines+markers', name=group)
        data.append(trace)
    trace = go.Scatter(x=[list(l['Time'])[0]], y=[list(l['fc'])[0]], marker=marker, mode='markers', name='Latest')
    data.append(trace)
    layout = go.Layout(xaxis={'title': 'Time'}, yaxis={'title': x.lower()}, width=450, height=250, showlegend=(False if x!='Error' else True), margin=dict(r=0, t=10, b=10))
    figure = go.Figure(data=data, layout=layout)
    return [figure]

##### Show the prediction of the latest data point, on the dashboard

In [ ]:
@app.callback([Output('prediction','children')], [Input('interval-component', 'n_intervals')])
def pred(n):
    global l, color
    return [html.H2(children=list(l['label'])[0], style={'textAlign': 'start', 'color':color})]

##### And the parameters of the latest data point (consumed by the Consumer), as a table

In [ ]:
@app.callback([Output('table','children')], [Input('interval-component', 'n_intervals')])
def tab(n):
    global l
    # Make a table of the values of the current data point
    values = [list(l[x])[0] for x in params]
    table_body = [html.Tbody([html.Tr([html.Td(x, style={'paddingTop': '3px', 'paddingBottom': '3px'}), html.Td(values[i], style={'paddingTop': '3px', 'paddingBottom': '3px'})]) for i,x in enumerate(params)])]
    table = dbc.Table(table_body, bordered=True, style={'width':300,'marginLeft': 'auto', 'marginRight': 'auto'})
    return [table]

##### Run the Dash app on localhost

In [ ]:
if __name__ == "__main__":
    app.run_server(host= '0.0.0.0', debug=False)